# Notebook 1: Classic Dual-Curve Bootstrap (2012)

This notebook implements the logic defined in the paper *The Past, Present and Future of Curves*.
We bootstrap an **OIS Discount Curve** first, then a **LIBOR Forward Curve** that depends on the OIS curve for discounting.


In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import QuantLib as ql
from src.framework import CalibrationTarget, InstrumentStrategy
from src.classic import ClassicModel

# Set Evaluation Date
calc_date = ql.Date(30, ql.October, 2014)
model = ClassicModel(calc_date)
print(f"Valuation Date set to: {calc_date}")


In [ ]:
# 1. Define Strategies (Data)

# OIS Instruments (Rate, Tenor, Type)
ois_instruments = [
    ('1D', 0.00090, 'Deposit'), 
    ('1Y', 0.001650, 'OIS'), 
    ('5Y', 0.015880, 'OIS'), 
    ('10Y', 0.022530, 'OIS'), 
    ('30Y', 0.027920, 'OIS')
]

ois_conv = {
    'settlement': 2, 
    'calendar': ql.UnitedStates(ql.UnitedStates.Settlement),
    'day_count': ql.Actual360(), 
    'index': ql.FedFunds()
}

ois_strat = InstrumentStrategy("USD_OIS_Strat", ois_instruments, ois_conv)

# Libor Instruments
libor_instruments = [
    ('3M', 0.0025, 'Deposit'), 
    ('1Y', 0.0045, 'Swap'), 
    ('5Y', 0.0175, 'Swap'), 
    ('10Y', 0.0240, 'Swap')
]

libor_conv = {
    'settlement': 2, 
    'calendar': ql.UnitedStates(ql.UnitedStates.Settlement),
    'day_count': ql.Actual360(), 
    'fixed_day_count': ql.Thirty360(ql.Thirty360.BondBasis)
}

libor_strat = InstrumentStrategy("USD_Libor_Strat", libor_instruments, libor_conv)


In [ ]:
# 2. Define Targets (Configuration)

target_ois = CalibrationTarget("USD_OIS", "LogLinear", "Discount")
target_libor = CalibrationTarget("USD_Libor3M", "LogLinear", "Forward", ql.USDLibor(ql.Period(3, ql.Months)))


In [ ]:
# 3. Calibrate (Execution)

# Build OIS
handle_ois = model.calibrate(target_ois, ois_strat)

# Build Libor (Injecting OIS as exogenous discount source)
handle_libor = model.calibrate(target_libor, libor_strat, exogenous_discount="USD_OIS")

# Validation
print(f"OIS DF at 10Y: {handle_ois.discount(10.0):.6f}")
print(f"Libor DF at 10Y: {handle_libor.discount(10.0):.6f}")
